In [1]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, accuracy_score
import csv

In [2]:
def get_algo_path():
    import os
    cwd = os.getcwd()
    s='\\'
    if '\\' not in cwd:
        s='/'
    cwd= cwd.split(s)[4:-1]
    cwd.append('Not Normalized')
    cwd='/'.join(cwd)
    return cwd

def get_csv_path():
    import os
    cwd = os.getcwd()
    s="\\"
    if "\\" not in cwd:
        s='/'
    file= cwd.split(s)[:4]
    file.append('models_scores.csv')
    file= s.join(file)
    return file

def line_is_exist(file, row):
    logfile = open(file, 'r')
    loglist = logfile.readlines()
    logfile.close()
    for line in loglist:
        if ','.join(row) in line:
            return True
    return False

def write_new_score(file, line):
    if( not line_is_exist(file, line) ):
        with open(file, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(line)
    else:
        print('line exsist already')
    

In [3]:
data= pd.read_csv('../data.csv')

In [4]:
##copying data
data1=data.copy()
### spliting data en X et Y
X= data1.drop('Loan Status', axis=1)
Y= data1['Loan Status']
### spliting the data to train and test
X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 123)
## dicision tree (CART)
dct = DecisionTreeClassifier()
params={'criterion' : ['gini', 'entropy'],
        'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14, 15, 16, 17, None]}

##accuracy
grid_search_acc= GridSearchCV(estimator=dct, param_grid= params, scoring='accuracy',cv=2, n_jobs=-1)
grid_search_acc= grid_search_acc.fit(X_train, Y_train)
y_predict= grid_search_acc.best_estimator_.predict(X_test)


In [5]:
## get avg precision & avg recall
report= classification_report(Y_test, y_predict, output_dict=True)
avg_list = report.pop("weighted avg")
avg_precision= round(avg_list['precision'], 3)
avg_recall= round(avg_list['recall'], 3)
accuraccy= round(accuracy_score(Y_test,y_predict), 3)
## csv row
csv_row = [get_algo_path(), 'Decision Tree', str(grid_search_acc.best_params_), str(accuraccy), str(avg_precision), str(avg_recall)]
## write file
csv_file= get_csv_path()
write_new_score(csv_file, csv_row)

In [6]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")


----------
best parameters:  {'criterion': 'gini', 'max_depth': 5}
----------
[[ 1266  5119]
 [    0 16906]]
----------
              precision    recall  f1-score   support

           0       1.00      0.20      0.33      6385
           1       0.77      1.00      0.87     16906

    accuracy                           0.78     23291
   macro avg       0.88      0.60      0.60     23291
weighted avg       0.83      0.78      0.72     23291

----------


In [7]:
grid_search_acc.cv_results_



{'mean_fit_time': array([0.18251193, 0.22838771, 0.25531757, 0.35654879, 0.38696563,
        0.40192306, 0.46774805, 0.45478535, 0.41937804, 0.43932521,
        0.52260256, 0.55750883, 0.54354632, 0.60238945, 0.70262003,
        0.2732687 , 0.3286202 , 0.33660054, 0.44331324, 0.47173929,
        0.5340718 , 0.5609988 , 0.61385918, 0.64427531, 0.64926374,
        0.7230649 , 0.77542567, 0.81033313, 0.85022652, 0.91405427]),
 'std_fit_time': array([0.00199568, 0.00797784, 0.0129658 , 0.01446128, 0.00398958,
        0.00099885, 0.00199306, 0.02792573, 0.01246667, 0.01346457,
        0.00099778, 0.01894963, 0.00897634, 0.02094471, 0.00149608,
        0.00199485, 0.00149703, 0.01047277, 0.01845014, 0.004987  ,
        0.02543175, 0.0124656 , 0.01545882, 0.00797927, 0.01695478,
        0.02393579, 0.01745391, 0.02044642, 0.01047122, 0.0184499 ]),
 'mean_score_time': array([0.01146996, 0.01196802, 0.01196706, 0.01994419, 0.01296544,
        0.01296556, 0.01346374, 0.01346278, 0.01396227, 0.02